In [1]:
import boto3
import io
import re
import numpy as np
import os
import xml.etree.ElementTree as ET 
from google.cloud import vision
import requests
from time import time
from utilities.score_computors import jaccard_similarity
from utilities.pythonDB import writeToDB, recordsExists

In [2]:
def extract_clean_str(original_str):
    original_str = original_str.replace('<br/>', '')
    return re.sub('[^a-zA-Z0-9]+', ' ', original_str, flags=re.UNICODE).lower().strip().replace(' ', '')

def get_mapped(path):
    key_map, dict_idx = {}, 0
    for r, _, f in os.walk(dir_path):
        for file in f:
            if '.JPG' in file or '.jpg' in file:
                key_map[dict_idx] = file
                dict_idx += 1
    idxs = list(key_map.keys())          
    shuffled_idxs = np.random.randint(0, len(idxs), len(idxs))
    return key_map, shuffled_idxs

def fetch_from_xml(path, file):
    tree = ET.parse(os.path.join(path, file)).getroot()  
    return extract_clean_str(''.join([e.attrib['char'] for e in tree.iter(tag='character')]))

In [3]:
class RegonizeText():
    def __init__(self, base, file): 
        with io.open(os.path.join(base, file), 'rb') as image_file:
            content = (image_file.read())
        self.content = content
        self.base, self.file, self.actual_str, self.detected_str = base, file, '', ''
        self.label = ''
        self.start = time()
        
    def return_function(self, name):
        dataset = dir_path.split('/')[1]
        if recordsExists(self.file, dataset, name):
            return
        
        self.label = name
        status, score =  getattr(self, 'if_' + name)()
        compute_time = time() - self.start
        bag = (self.file, dataset, self.label, status, score, compute_time, self.actual_str, self.detected_str)
        writeToDB(bag)
    def compute_ground_truth(self):
        ''' 
            Status = {-1: 'XML File Missing', -2: 'Error in XML', -3: 'Ground Truth Empty', -4: 'API Error', 0: 'All Correct'} 
            Return = Ground Truth, Status
        
        '''
        xml_file = self.file[:len(self.file)-4] + '.xml' 
        
        if not os.path.exists(os.path.join(self.base, self.file)):
            return '', -1
        try:
            ground_truth_str = fetch_from_xml(dir_path, xml_file)
        except: 
            return '', -2
        
        if len(ground_truth_str) == 0:
            return '', -3
        else:
            self.actual_str = ground_truth_str
            return ground_truth_str, 0

    def if_aws(self):
        ground_truth, xml_status = self.compute_ground_truth()
        if xml_status != 0: return xml_status, 0
        
        try:
            imgobj = {'Bytes': self.content}
            client=boto3.client('rekognition', region_name='us-east-1')
            response=client.detect_text(Image=imgobj)
            detected_str = extract_clean_str(''.join([txt['DetectedText'] for txt in response['TextDetections'] \
                                                   if txt['Type']=='WORD']))
            self.detected_str = detected_str
            score = jaccard_similarity(detected_str, ground_truth)
            return 0, score
        except:
            return -4, 0

    def if_gc(self):
        ground_truth, xml_status = self.compute_ground_truth()
        if xml_status != 0: return xml_status, 0
        try:
            vision_client = vision.ImageAnnotatorClient()
            image = vision.types.Image(content=self.content)
            text_detection_response = vision_client.text_detection(image=image)
            detected_str = extract_clean_str(text_detection_response.full_text_annotation.text)
            self.detected_str = detected_str
            score = jaccard_similarity(detected_str, ground_truth)
            return 0, score
        except:
            return -4, 0
    
    def if_azure(self):
        ground_truth, xml_status = self.compute_ground_truth()
        if xml_status != 0: return xml_status, 0
        try:
            subscription_key, endpoint = os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY'], os.environ['COMPUTER_VISION_ENDPOINT']
            ocr_url = endpoint + "vision/v2.1/ocr"
            headers = {'Ocp-Apim-Subscription-Key': subscription_key, 'Content-Type': 'application/octet-stream'}
            params = {'detectOrientation': 'true'}
            response = requests.post(ocr_url, headers=headers, params=params, data=self.content)
            response.raise_for_status()

            lines_info = [region["lines"] for region in response.json()["regions"]]
            detected_str = extract_clean_str(''.join([word_info['text'] for line in lines_info \
                                      for word_metadata in line for word_info in word_metadata["words"]]))
            self.detected_str = detected_str
            score = jaccard_similarity(detected_str, ground_truth)
            return 0, score
        except:
            return -4, 0

In [4]:
dir_path = 'datasets/KAIST/English/'
dict_files, files_idx = get_mapped(dir_path)
shuffled_idx = np.random.randint(0, len(files_idx), len(files_idx))

In [5]:
for idx in shuffled_idx:
    file_name = dict_files.get(idx)
    print ("Iterating for File Name = {}".format(file_name))
    RegonizeText(dir_path, file_name).return_function('azure')

Iterating for File Name = DSC03409.JPG
Iterating for File Name = DSC03402.JPG
Iterating for File Name = P090911086.jpg
Iterating for File Name = DSC02910.JPG
Iterating for File Name = DSC02897.JPG
Iterating for File Name = DSC02962.JPG
Iterating for File Name = DSC03556.JPG
Iterating for File Name = DSC03217.JPG
Iterating for File Name = DSC03047.JPG
Iterating for File Name = DSC02879.JPG
Iterating for File Name = DSC04321.JPG
Iterating for File Name = DSC03311.JPG
Iterating for File Name = DSC04223.JPG
Iterating for File Name = DSC03192.JPG
Iterating for File Name = DSC03643.JPG
Iterating for File Name = P090908006.jpg
Iterating for File Name = P090831010.jpg
Iterating for File Name = DSC03737.JPG
Iterating for File Name = IMG_2618.JPG
Iterating for File Name = DSC04456.JPG
Iterating for File Name = DSC03579.JPG
Iterating for File Name = DSC02756.JPG
Iterating for File Name = 049.JPG
Iterating for File Name = DSC03185.JPG
Iterating for File Name = DSC03517.JPG
Iterating for File Name 

Iterating for File Name = DSC03682.JPG
Iterating for File Name = 029.jpg
Iterating for File Name = DSC04125.JPG
Iterating for File Name = DSC02936.JPG
Iterating for File Name = P1010114.JPG
Iterating for File Name = P090831115.jpg
Iterating for File Name = DSC03331.JPG
Iterating for File Name = 2007-12-08-day-015.jpg
Iterating for File Name = P090903068.jpg
Iterating for File Name = DSC02319.JPG
Iterating for File Name = DSC02854.JPG
Iterating for File Name = 078.JPG
Iterating for File Name = DSC03414.JPG
Iterating for File Name = DSC02429.JPG
Iterating for File Name = DSC02391.JPG
Iterating for File Name = DSC02914.JPG
Iterating for File Name = DSC03533.JPG
Iterating for File Name = DSC04029.JPG
Iterating for File Name = P090903015.jpg
Iterating for File Name = DSC03395.JPG
Iterating for File Name = DSC04305.JPG
Iterating for File Name = DSC03501.JPG
Iterating for File Name = P090831005.jpg
Iterating for File Name = 129.JPG
Iterating for File Name = P090831007.jpg
Iterating for File N

Iterating for File Name = DSC02501.JPG
Iterating for File Name = DSC03756.JPG
Iterating for File Name = DSC04032.JPG
Iterating for File Name = DSC02243.JPG
Iterating for File Name = DSC02492.JPG
Iterating for File Name = DSC04364.JPG
Iterating for File Name = DSC03820.JPG
Iterating for File Name = 082.JPG
Iterating for File Name = DSC02471.JPG
Iterating for File Name = DSC02842.JPG
Iterating for File Name = DSC03829.JPG
Iterating for File Name = DSC04183.JPG
Iterating for File Name = 080116-0066.jpg
Iterating for File Name = 027.jpg
Iterating for File Name = DSC03134.JPG
Iterating for File Name = DSC03705.JPG
Iterating for File Name = DSC03300.JPG
Iterating for File Name = DSC03586.JPG
Iterating for File Name = DSC04223.JPG
Iterating for File Name = DSC03123.JPG
Iterating for File Name = DSC03467.JPG
Iterating for File Name = DSC04065.JPG
Iterating for File Name = 031.JPG
Iterating for File Name = P1010080.JPG
Iterating for File Name = DSC02993.JPG
Iterating for File Name = m.jpg
Itera

Iterating for File Name = DSC04203.JPG
Iterating for File Name = P090903099.jpg
Iterating for File Name = DSC02770.JPG
Iterating for File Name = P1010129.JPG
Iterating for File Name = DSC03267.JPG
Iterating for File Name = P090911016.jpg
Iterating for File Name = DSC03090.JPG
Iterating for File Name = DSC02795.JPG
Iterating for File Name = DSC03901.JPG
Iterating for File Name = DSC02954.JPG
Iterating for File Name = DSC04094.JPG
Iterating for File Name = P090831014.jpg
Iterating for File Name = DSC03274.JPG
Iterating for File Name = DSC02750.JPG
Iterating for File Name = DSC02501.JPG
Iterating for File Name = DSC03441.JPG
Iterating for File Name = DSC03757.JPG
Iterating for File Name = DSC03759.JPG
Iterating for File Name = DSC04364.JPG
Iterating for File Name = DSC02678.JPG
Iterating for File Name = DSC03199.JPG
Iterating for File Name = 133.JPG
Iterating for File Name = DSC04059.JPG
Iterating for File Name = DSC02410.JPG
Iterating for File Name = 55.JPG
Iterating for File Name = DSC0

Iterating for File Name = DSC02886.JPG
Iterating for File Name = P090831136.jpg
Iterating for File Name = P090831115.jpg
Iterating for File Name = DSC04312.JPG
Iterating for File Name = DSC04447.JPG
Iterating for File Name = P1010075.JPG
Iterating for File Name = DSC02357.JPG
Iterating for File Name = DSC02502.JPG
Iterating for File Name = P1010073.JPG
Iterating for File Name = DSC03761.JPG
Iterating for File Name = DSC03046.JPG
Iterating for File Name = DSC03328.JPG
Iterating for File Name = DSC03667.JPG
Iterating for File Name = DSC03047.JPG
Iterating for File Name = DSC04155.JPG
Iterating for File Name = DSC02933.JPG
Iterating for File Name = 2007-12-08-day-067.jpg
Iterating for File Name = P090831045.jpg
Iterating for File Name = DSC04072.JPG
Iterating for File Name = DSC04464.JPG
Iterating for File Name = P1010030.JPG
Iterating for File Name = DSC04361.JPG
Iterating for File Name = 9.JPG
Iterating for File Name = DSC02884.JPG
Iterating for File Name = P1010096.JPG
Iterating for Fi

Iterating for File Name = 046.JPG
Iterating for File Name = DSC02644.JPG
Iterating for File Name = DSC02243.JPG
Iterating for File Name = P1010002.JPG
Iterating for File Name = DSC04009.JPG
Iterating for File Name = 001.jpg
Iterating for File Name = DSC03258.JPG
Iterating for File Name = DSC03415.JPG
Iterating for File Name = DSC02840.JPG
Iterating for File Name = DSC03561.JPG
Iterating for File Name = DSC03249.JPG
Iterating for File Name = 45.JPG
Iterating for File Name = DSC03830.JPG
Iterating for File Name = 15.JPG
Iterating for File Name = P090911084.jpg
Iterating for File Name = DSC03758.JPG
Iterating for File Name = DSC03197.JPG
Iterating for File Name = 003.JPG
Iterating for File Name = DSC03012.JPG
Iterating for File Name = P090911064.jpg
Iterating for File Name = DSC03254.JPG
Iterating for File Name = DSC02532.JPG
Iterating for File Name = DSC02664.JPG
Iterating for File Name = DSC03303.JPG
Iterating for File Name = DSC04458.JPG
Iterating for File Name = P090831122.jpg
Iterati

In [6]:
for idx in shuffled_idx:
    file_name = dict_files.get(idx)
    print ("Iterating for File Name = {}".format(file_name))
    RegonizeText(dir_path, file_name).return_function('aws')

Iterating for File Name = DSC03409.JPG
Iterating for File Name = DSC03402.JPG
Iterating for File Name = P090911086.jpg
Iterating for File Name = DSC02910.JPG
Iterating for File Name = DSC02897.JPG
Iterating for File Name = DSC02962.JPG
Iterating for File Name = DSC03556.JPG
Iterating for File Name = DSC03217.JPG
Iterating for File Name = DSC03047.JPG
Iterating for File Name = DSC02879.JPG
Iterating for File Name = DSC04321.JPG
Iterating for File Name = DSC03311.JPG
Iterating for File Name = DSC04223.JPG
Iterating for File Name = DSC03192.JPG
Iterating for File Name = DSC03643.JPG
Iterating for File Name = P090908006.jpg
Iterating for File Name = P090831010.jpg
Iterating for File Name = DSC03737.JPG
Iterating for File Name = IMG_2618.JPG
Iterating for File Name = DSC04456.JPG
Iterating for File Name = DSC03579.JPG
Iterating for File Name = DSC02756.JPG
Iterating for File Name = 049.JPG
Iterating for File Name = DSC03185.JPG
Iterating for File Name = DSC03517.JPG
Iterating for File Name 

Iterating for File Name = P090831124.jpg
Iterating for File Name = DSC04048.JPG
Iterating for File Name = DSC04146.JPG
Iterating for File Name = DSC04119.JPG
Iterating for File Name = DSC02572.JPG
Iterating for File Name = DSC03177.JPG
Iterating for File Name = DSC03682.JPG
Iterating for File Name = 029.jpg
Iterating for File Name = DSC04125.JPG
Iterating for File Name = DSC02936.JPG
Iterating for File Name = P1010114.JPG
Iterating for File Name = P090831115.jpg
Iterating for File Name = DSC03331.JPG
Iterating for File Name = 2007-12-08-day-015.jpg
Iterating for File Name = P090903068.jpg
Iterating for File Name = DSC02319.JPG
Iterating for File Name = DSC02854.JPG
Iterating for File Name = 078.JPG
Iterating for File Name = DSC03414.JPG
Iterating for File Name = DSC02429.JPG
Iterating for File Name = DSC02391.JPG
Iterating for File Name = DSC02914.JPG
Iterating for File Name = DSC03533.JPG
Iterating for File Name = DSC04029.JPG
Iterating for File Name = P090903015.jpg
Iterating for Fil

Iterating for File Name = DSC02675.JPG
Iterating for File Name = 080118-0007.jpg
Iterating for File Name = DSC03687.JPG
Iterating for File Name = DSC02917.JPG
Iterating for File Name = DSC02398.JPG
Iterating for File Name = DSC03150.JPG
Iterating for File Name = DSC04140.JPG
Iterating for File Name = DSC03731.JPG
Iterating for File Name = DSC02501.JPG
Iterating for File Name = DSC03756.JPG
Iterating for File Name = DSC04032.JPG
Iterating for File Name = DSC02243.JPG
Iterating for File Name = DSC02492.JPG
Iterating for File Name = DSC04364.JPG
Iterating for File Name = DSC03820.JPG
Iterating for File Name = 082.JPG
Iterating for File Name = DSC02471.JPG
Iterating for File Name = DSC02842.JPG
Iterating for File Name = DSC03829.JPG
Iterating for File Name = DSC04183.JPG
Iterating for File Name = 080116-0066.jpg
Iterating for File Name = 027.jpg
Iterating for File Name = DSC03134.JPG
Iterating for File Name = DSC03705.JPG
Iterating for File Name = DSC03300.JPG
Iterating for File Name = DSC

Iterating for File Name = P090831119.jpg
Iterating for File Name = DSC03185.JPG
Iterating for File Name = DSC03822.JPG
Iterating for File Name = DSC02879.JPG
Iterating for File Name = P1010133.JPG
Iterating for File Name = DSC03291.JPG
Iterating for File Name = DSC04421.JPG
Iterating for File Name = DSC02673.JPG
Iterating for File Name = DSC04203.JPG
Iterating for File Name = P090903099.jpg
Iterating for File Name = DSC02770.JPG
Iterating for File Name = P1010129.JPG
Iterating for File Name = DSC03267.JPG
Iterating for File Name = P090911016.jpg
Iterating for File Name = DSC03090.JPG
Iterating for File Name = DSC02795.JPG
Iterating for File Name = DSC03901.JPG
Iterating for File Name = DSC02954.JPG
Iterating for File Name = DSC04094.JPG
Iterating for File Name = P090831014.jpg
Iterating for File Name = DSC03274.JPG
Iterating for File Name = DSC02750.JPG
Iterating for File Name = DSC02501.JPG
Iterating for File Name = DSC03441.JPG
Iterating for File Name = DSC03757.JPG
Iterating for Fil

Iterating for File Name = DSC03773.JPG
Iterating for File Name = DSC04312.JPG
Iterating for File Name = DSC03062.JPG
Iterating for File Name = DSC03365.JPG
Iterating for File Name = DSC03582.JPG
Iterating for File Name = P1010064.JPG
Iterating for File Name = DSC04032.JPG
Iterating for File Name = DSC02646.JPG
Iterating for File Name = DSC03660.JPG
Iterating for File Name = 031.JPG
Iterating for File Name = DSC02769.JPG
Iterating for File Name = DSC03187.JPG
Iterating for File Name = DSC03223.JPG
Iterating for File Name = 080119-0009.jpg
Iterating for File Name = DSC02713.JPG
Iterating for File Name = DSC03254.JPG
Iterating for File Name = DSC02240.JPG
Iterating for File Name = DSC04160.JPG
Iterating for File Name = P090905003.jpg
Iterating for File Name = DSC04290.JPG
Iterating for File Name = P1010103.JPG
Iterating for File Name = DSC02780.JPG
Iterating for File Name = P1010069.JPG
Iterating for File Name = DSC03656.JPG
Iterating for File Name = DSC03633.JPG
Iterating for File Name =

Iterating for File Name = 105.jpg
Iterating for File Name = DSC03067.JPG
Iterating for File Name = DSC03942.JPG
Iterating for File Name = DSC03469.JPG
Iterating for File Name = P1010087.JPG
Iterating for File Name = DSC03865.JPG
Iterating for File Name = DSC03119.JPG
Iterating for File Name = DSC02570.JPG
Iterating for File Name = P090903092.jpg
Iterating for File Name = DSC02893.JPG
Iterating for File Name = DSC04087.JPG
Iterating for File Name = DSC02789.JPG
Iterating for File Name = 049.JPG
Iterating for File Name = DSC03301.JPG
Iterating for File Name = DSC04135.JPG
Iterating for File Name = DSC03093.JPG
Iterating for File Name = DSC03203.JPG
Iterating for File Name = 080119-0016.jpg
Iterating for File Name = DSC03129.JPG
Iterating for File Name = P090903050.jpg
Iterating for File Name = DSC03604.JPG
Iterating for File Name = P1010030.JPG
Iterating for File Name = DSC04019.JPG
Iterating for File Name = DSC03312.JPG
Iterating for File Name = DSC03309.JPG
Iterating for File Name = DS

In [7]:
for idx in shuffled_idx:
    file_name = dict_files.get(idx)
    print ("Iterating for File Name = {}".format(file_name))
    RegonizeText(dir_path, file_name).return_function('gc')

Iterating for File Name = DSC03409.JPG
Iterating for File Name = DSC03402.JPG
Iterating for File Name = P090911086.jpg
Iterating for File Name = DSC02910.JPG
Iterating for File Name = DSC02897.JPG
Iterating for File Name = DSC02962.JPG
Iterating for File Name = DSC03556.JPG
Iterating for File Name = DSC03217.JPG
Iterating for File Name = DSC03047.JPG
Iterating for File Name = DSC02879.JPG
Iterating for File Name = DSC04321.JPG
Iterating for File Name = DSC03311.JPG
Iterating for File Name = DSC04223.JPG
Iterating for File Name = DSC03192.JPG
Iterating for File Name = DSC03643.JPG
Iterating for File Name = P090908006.jpg
Iterating for File Name = P090831010.jpg
Iterating for File Name = DSC03737.JPG
Iterating for File Name = IMG_2618.JPG
Iterating for File Name = DSC04456.JPG
Iterating for File Name = DSC03579.JPG
Iterating for File Name = DSC02756.JPG
Iterating for File Name = 049.JPG
Iterating for File Name = DSC03185.JPG
Iterating for File Name = DSC03517.JPG
Iterating for File Name 

Iterating for File Name = P090831124.jpg
Iterating for File Name = DSC04048.JPG
Iterating for File Name = DSC04146.JPG
Iterating for File Name = DSC04119.JPG
Iterating for File Name = DSC02572.JPG
Iterating for File Name = DSC03177.JPG
Iterating for File Name = DSC03682.JPG
Iterating for File Name = 029.jpg
Iterating for File Name = DSC04125.JPG
Iterating for File Name = DSC02936.JPG
Iterating for File Name = P1010114.JPG
Iterating for File Name = P090831115.jpg
Iterating for File Name = DSC03331.JPG
Iterating for File Name = 2007-12-08-day-015.jpg
Iterating for File Name = P090903068.jpg
Iterating for File Name = DSC02319.JPG
Iterating for File Name = DSC02854.JPG
Iterating for File Name = 078.JPG
Iterating for File Name = DSC03414.JPG
Iterating for File Name = DSC02429.JPG
Iterating for File Name = DSC02391.JPG
Iterating for File Name = DSC02914.JPG
Iterating for File Name = DSC03533.JPG
Iterating for File Name = DSC04029.JPG
Iterating for File Name = P090903015.jpg
Iterating for Fil

Iterating for File Name = DSC02675.JPG
Iterating for File Name = 080118-0007.jpg
Iterating for File Name = DSC03687.JPG
Iterating for File Name = DSC02917.JPG
Iterating for File Name = DSC02398.JPG
Iterating for File Name = DSC03150.JPG
Iterating for File Name = DSC04140.JPG
Iterating for File Name = DSC03731.JPG
Iterating for File Name = DSC02501.JPG
Iterating for File Name = DSC03756.JPG
Iterating for File Name = DSC04032.JPG
Iterating for File Name = DSC02243.JPG
Iterating for File Name = DSC02492.JPG
Iterating for File Name = DSC04364.JPG
Iterating for File Name = DSC03820.JPG
Iterating for File Name = 082.JPG
Iterating for File Name = DSC02471.JPG
Iterating for File Name = DSC02842.JPG
Iterating for File Name = DSC03829.JPG
Iterating for File Name = DSC04183.JPG
Iterating for File Name = 080116-0066.jpg
Iterating for File Name = 027.jpg
Iterating for File Name = DSC03134.JPG
Iterating for File Name = DSC03705.JPG
Iterating for File Name = DSC03300.JPG
Iterating for File Name = DSC

Iterating for File Name = P090831119.jpg
Iterating for File Name = DSC03185.JPG
Iterating for File Name = DSC03822.JPG
Iterating for File Name = DSC02879.JPG
Iterating for File Name = P1010133.JPG
Iterating for File Name = DSC03291.JPG
Iterating for File Name = DSC04421.JPG
Iterating for File Name = DSC02673.JPG
Iterating for File Name = DSC04203.JPG
Iterating for File Name = P090903099.jpg
Iterating for File Name = DSC02770.JPG
Iterating for File Name = P1010129.JPG
Iterating for File Name = DSC03267.JPG
Iterating for File Name = P090911016.jpg
Iterating for File Name = DSC03090.JPG
Iterating for File Name = DSC02795.JPG
Iterating for File Name = DSC03901.JPG
Iterating for File Name = DSC02954.JPG
Iterating for File Name = DSC04094.JPG
Iterating for File Name = P090831014.jpg
Iterating for File Name = DSC03274.JPG
Iterating for File Name = DSC02750.JPG
Iterating for File Name = DSC02501.JPG
Iterating for File Name = DSC03441.JPG
Iterating for File Name = DSC03757.JPG
Iterating for Fil

Iterating for File Name = DSC03773.JPG
Iterating for File Name = DSC04312.JPG
Iterating for File Name = DSC03062.JPG
Iterating for File Name = DSC03365.JPG
Iterating for File Name = DSC03582.JPG
Iterating for File Name = P1010064.JPG
Iterating for File Name = DSC04032.JPG
Iterating for File Name = DSC02646.JPG
Iterating for File Name = DSC03660.JPG
Iterating for File Name = 031.JPG
Iterating for File Name = DSC02769.JPG
Iterating for File Name = DSC03187.JPG
Iterating for File Name = DSC03223.JPG
Iterating for File Name = 080119-0009.jpg
Iterating for File Name = DSC02713.JPG
Iterating for File Name = DSC03254.JPG
Iterating for File Name = DSC02240.JPG
Iterating for File Name = DSC04160.JPG
Iterating for File Name = P090905003.jpg
Iterating for File Name = DSC04290.JPG
Iterating for File Name = P1010103.JPG
Iterating for File Name = DSC02780.JPG
Iterating for File Name = P1010069.JPG
Iterating for File Name = DSC03656.JPG
Iterating for File Name = DSC03633.JPG
Iterating for File Name =

Iterating for File Name = 105.jpg
Iterating for File Name = DSC03067.JPG
Iterating for File Name = DSC03942.JPG
Iterating for File Name = DSC03469.JPG
Iterating for File Name = P1010087.JPG
Iterating for File Name = DSC03865.JPG
Iterating for File Name = DSC03119.JPG
Iterating for File Name = DSC02570.JPG
Iterating for File Name = P090903092.jpg
Iterating for File Name = DSC02893.JPG
Iterating for File Name = DSC04087.JPG
Iterating for File Name = DSC02789.JPG
Iterating for File Name = 049.JPG
Iterating for File Name = DSC03301.JPG
Iterating for File Name = DSC04135.JPG
Iterating for File Name = DSC03093.JPG
Iterating for File Name = DSC03203.JPG
Iterating for File Name = 080119-0016.jpg
Iterating for File Name = DSC03129.JPG
Iterating for File Name = P090903050.jpg
Iterating for File Name = DSC03604.JPG
Iterating for File Name = P1010030.JPG
Iterating for File Name = DSC04019.JPG
Iterating for File Name = DSC03312.JPG
Iterating for File Name = DSC03309.JPG
Iterating for File Name = DS